## Test des modeles (cm,roc,...)

In [1]:
def list_to_txt(lst, filename):
    # Sauvegarde directement les valeurs (sans normaliser)
    with open(filename, 'w', encoding='utf-8') as f:
        for value in lst:
            f.write(f"{value}\n")

In [54]:
import numpy as np
import pickle
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc
import seaborn as sns
from matplotlib import pyplot as plt
from importlib import reload
import metric_seq2seq
reload(metric_seq2seq)
from metric_seq2seq import *
import pickle
import os

%matplotlib qt

#choix du modéle utilisé
mic = "12"         #le microphone souhaité à tester
feature = "audio"
dossier = "acceleros" if feature == "acc" else "audios"
file = f"{feature}_{mic}.pkl"
dossier = "fusion"
file = "acc134_aud1234.pkl"
file_donnees = os.path.join("models",dossier,file)

print(file_donnees)
with open(file_donnees, "rb") as f:
    data0 = pickle.load(f)

f1s = []
y_true_all = []
y_pred_all = []

for i in [0,1,2]:
    y_true_data , y_pred_data = data0[i]
    y_true = y_true_data.argmax(axis=-1).flatten()
    y_pred = (y_pred_data[:,1] > 0.5).astype(int)
    y_true_all.append(y_true)
    y_pred_all.append(y_pred)
    f1s.append(f1_score(y_true, y_pred, average="binary", pos_label=1))

'''plt.figure(figsize=(15,8))
plt.step(range(len(y_true)),y_true,label="true")
plt.step(range(len(y_true)),y_pred,label="pred",linestyle="--")

#plt.step(range(len(y_true)),y_pred_data[:,1],label="pred")
plt.legend()
plt.show()'''

sets = ["Train", "Val", "Test"]
plt.figure(figsize=(15,8))

if True:
    cm = confusion_matrix(y_true_all[1], y_pred_all[1])
    cm_normalized = cm.astype(float) / cm.sum(axis=1, keepdims=True)

    # Création des annotations : "xx.xx%\n(nb)"
    annotations = np.empty_like(cm, dtype=object)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            annotations[i, j] = f"{cm_normalized[i, j]*100:.2f}%\n({cm[i, j]})"

    sns.heatmap(cm_normalized, annot=annotations, fmt="", cmap="Blues")
    plt.title(f"{sets[2]} Confusion Matrix des données de test {mic}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

history = data0[3][2]

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

metrics = ["loss", "f1_class_1", "auc_binary"]

for i in range(2):
    for j in range(2):
        if i ==1 and j ==1 :
            break
        metric = metrics[i * 2 + j]  
        
        axes[i, j].plot(history[metric], label="train")
        axes[i, j].plot(history[f"val_{metric}"], label="val")
        
        axes[i, j].set_title(metric)
        axes[i, j].legend()

plt.tight_layout()
plt.show()


models\fusion\acc134_aud1234.pkl


c:\Users\mohammed-amine.tanna\AppData\Local\anaconda3\envs\fft_env\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
plt.figure(figsize=(6,4))
plt.bar(["train","val","test"], f1s, color=["skyblue","orange","green"])
plt.title("F1 Score")
plt.show()


sets = ["Train", "Val", "Test"]

if True:
    cm = confusion_matrix(y_true_all[1], y_pred_all[1])
    cm_normalized = cm.astype("float") / cm.sum(axis=1, keepdims=True)
    
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues")
    plt.title(f"{sets[2]} Confusion Matrix des données de test {mic}'")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


plt.figure(figsize=(7,7))
for i in range(3):
    y_true = y_true_all[i]
    y_pred = y_pred_all[i]
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{sets[i]} AUC={roc_auc:.2f}")

plt.plot([0,1],[0,1],"k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC Curves micro {mic}")
plt.legend()
plt.show()


history = data0[3][2]

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

metrics = ["loss", "f1_class_1", "auc_binary"]

for i in range(2):
    for j in range(2):
        if i ==1 and j ==1 :
            break
        metric = metrics[i * 2 + j]  
        
        axes[i, j].plot(history[metric], label="train")
        axes[i, j].plot(history[f"val_{metric}"], label="val")
        
        axes[i, j].set_title(metric)
        axes[i, j].legend()

plt.tight_layout()
plt.show()

## Prediction sur les données d'aller

In [7]:
from tensorflow.keras.models import load_model
from metric_seq2seq import AUCSeq, F1ScoreSeq, PrecisionSeq, RecallSeq
import pickle
import os

custom_objects = {
    "AUCSeq": AUCSeq,
    "F1ScoreSeq": F1ScoreSeq,
    "PrecisionSeq": PrecisionSeq,
    "RecallSeq": RecallSeq
}

dossier = "audios"
file = f'{"audio"}_{12}'
'''file = f'acc34_aud12'
dossier = "fusion"'''
model_file = os.path.join('models',dossier,file+'.keras')
model = load_model(
    model_file,
    custom_objects=custom_objects,compile=False
)

data_model_pkl = os.path.join('models',dossier,file+'.pkl')
with open(data_model_pkl,"rb") as f:
    data = pickle.load(f)

scaler = data[3][1]
print(type(scaler))
#model.summary()

<class 'sklearn.preprocessing._data.StandardScaler'>


In [8]:
import random
random.seed(0)

import os 
from importlib import reload
import functions as fct
reload(fct)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import metric_seq2seq
reload(metric_seq2seq)
from metric_seq2seq import PrecisionSeq, RecallSeq, AUCSeq, F1ScoreSeq

debuts , fins = int(1*60+00),int(17*60+40)          # debut et fin en secondes
mf4 = 3
total = False
sr = 100000
tfn = 0.02
FMAX = 22000
seq2seq = False
N_window = fct.get_near_pof2(tfn,sr)
NB_FEATURES = 64
apply_pca = False
    #extraire les coefs mels
if True:  
    mels ,labels = [],[]
    for ind_train,debut,fin in zip([mf4],[debuts],[fins]):
        if  not total:
            xi,yi = fct.mels_features(list_mf4 = [ind_train],
                                        list_mics = [1,2],
                                        N_window = N_window,
                                        nb_mels = NB_FEATURES,
                                        power = 2.0,
                                        debut = debut,
                                        fin = fin,
                                        annot = True,
                                        sous_mean = True,
                                        feature = "audio",
                                        f_max = FMAX,
                                        overlap = 2, #25% de hop_lenght pour le calule des mels
                                        energie = "both",
                                        precoce = True,
                                        derive1 =True,
                                        derive2 = True,
                                        
                                        nbr_classes= 2 )
        
            
        elif total :
            mels ,labels = [],[] 
            for ind_train,debut,fin in zip([mf4],[debuts],[fins]):    #zip([ind_mf4_train],[int(1*60)],[int(17*60+40)])
                xi,yi = fct.mels_features(list_mf4 = [ind_train],
                                            list_mics = [1,2],
                                            N_window = N_window,
                                            debut = debut,
                                            fin = fin,
                                            annot = True,
                                            sous_mean = True,
                                            feature = "audio",
                                            f_max = 22000,
                                            overlap = 2, 
                                            energie = "both",
                                            precoce = True,
                                            power=2.0,
                                            nb_mels=64,
                                            derive1=True,
                                            derive2=False,
                                            nbr_classes= 2  )
                mels.append(xi)
                labels.append(yi)
            
            for ind_train,debut,fin in zip([mf4],[debuts],[fins]):    #zip([ind_mf4_train],[int(1*60)],[int(17*60+40)])
                xi,yi = fct.mels_features(list_mf4 = [ind_train],
                                            list_mics = [3],
                                            N_window = N_window,
                                            debut = debut,
                                            fin = fin,
                                            annot = True,
                                            sous_mean = True,
                                            feature = "acc",
                                            f_max = 10000,
                                            overlap = 2, 
                                            energie = "both",
                                            precoce = True,
                                            power=2.0,
                                            nb_mels=64,
                                            derive1=True,
                                            derive2=False,                                                                                       
                                            nbr_classes= 2 )
                mels.append(xi)
                labels.append(yi)


            X,Y = [],[]
            X_val,Y_val = [],[]
            X_test,Y_test = [],[]

            for key in mels[0].keys():
                X,Y = np.array(mels[0][key]) , np.array(labels[0][key])
            for key in mels[1].keys():
                X,Y = np.concatenate((X,mels[1][key]),axis=1),Y
            

            print(f"shape des données est {X.shape} et la distribution des classes esr {np.bincount(Y)}")


        if not total : 
            mels.append(xi)
            labels.append(yi)


            X,Y = [],[]
            X_val,Y_val = [],[]
            X_test,Y_test = [],[]

            for key in mels[0].keys():
                X,Y = np.array(mels[0][key]),np.array(labels[0][key])





    #prétraitement des donnees
    X,Y,_,_,_ = fct.pretrait(X,Y,n_components=0.95,
                                                apply_norm=True,
                                                seq=True,
                                                seq2seq=seq2seq,
                                                seq_apres= 10,
                                                seq_before= 9,
                                                apply_pca=apply_pca,
                                                overlap= 0.5,
                                                super_vecteur=False,
                                                scaler=scaler
                                                )


Normalisation par rapport au dataset est appliquée aux données de validation
8343 séquences créées avec un chevauchement de 50.0%.
Forme après création de séquences : (8343, 20, 388)
Poids de classe calculés [0.52657157 9.90855107]


In [9]:
y_pred_proba = model.predict(X)
y_pred = y_pred_proba.argmax(axis=1)

261/261 ━━━━━━━━━━━━━━━━━━━━ 250s 953ms/step


In [10]:
%matplotlib qt
import matplotlib.pyplot as plt
yp_probas = y_pred_proba[:,1]
'''list_to_txt(yp_probas,"pred.txt")
list_to_txt(Y,"true.txt")'''
y_pred =(yp_probas > 0.38).astype(int)
plt.figure(figsize=(25,5))
plt.step(range(len(Y)), Y, label="true")
#plt.step(range(len(Y)), y_pred, label="pred", linestyle='--')
plt.step(range(len(Y)), y_pred_proba[:,1], label="pred", linestyle='--')
plt.legend()
plt.show()

## Faire des comparaison entre differents models 

In [58]:
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.metrics import confusion_matrix,roc_curve,auc
import os


# Charger les fichiers pickle
models_name = ["audio_1","audio_2","acc_3","audio_12","acc_34","acc3_aud12","acc134_aud1234"]        #indices des modeles à comparer (12 : microphones 1 et 2 )
dossiers = ["audios","audios","acceleros","audios","acceleros","fusion","fusion"]
datas = []
files = []
for i,mic in enumerate(models_name) : 
    files.append(os.path.join('models',dossiers[i],f'{mic}.pkl'))

for i in range(len(models_name)) : 
    with open(files[i],"rb") as f : 
        datas.append(pickle.load(f))


# Fonction pour calculer le F1 score pour chaque ensemble
def compute_f1(data):
    sets = ['train', 'val', 'test']
    f1s = []
    for i, name in enumerate(sets):
        y_true = data[i][0].argmax(axis=1)
        y_pred = (data[i][1][:,1] > 0.5).astype(int)
        f1 = f1_score(y_true, y_pred, average="binary", pos_label=1)
        f1s.append((name, f1))
    return f1s

# Récupérer les scores
scores = []
for data in datas : 
    scores.append(compute_f1(data))


# Mettre les résultats dans un DataFrame
all_data = []

for model_name, scores in zip(
    [f"Model {i}" for i in models_name], 
    [*scores]
):
    for set_name, f1 in scores:
        all_data.append({'Model': model_name, 'Set': set_name, 'F1-score': f1})

df = pd.DataFrame(all_data)

# Tracer le graphique
plt.figure(figsize=(14,8))
sns.barplot(data=df, x='Set', y='F1-score', hue='Model')
plt.title('F1-score de la classe joint')
plt.ylim(0, 1)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.legend()
plt.show()
'''

fig , axes = plt.subplots(1,len(datas),figsize=(20,5))
for  data , axe , i in zip([*datas],axes,range(len(datas))) : 
    cm = confusion_matrix(data[2][0].argmax(axis=1) , data[2][1].argmax(axis=1))
    sns.heatmap(cm/cm.sum(axis=1,keepdims=True), annot=True,cmap='Blues', ax=axe)
    axe.set_title(f"Model {models_name[i]}")
    axe.set_xlabel('Prédiction')
    axe.set_ylabel('Réel')

model_names = [f"Model {i}" for i in range(len(datas))]
set_names = ['Train', 'Validation', 'Test']
set_indices = [0, 1, 2]

# Créer les sous-figures
fig, axes = plt.subplots(1, len(datas), figsize=(20, 5))

# Boucle sur les modèles et les axes
for data, axe, model_name in zip([*datas], axes, model_names):
    # Pour chaque ensemble (train, val, test)
    for idx, set_name in zip(set_indices, set_names):
        y_true = data[idx][0].argmax(axis=1)
        y_score = data[idx][1][:, 1]  # Probabilité de la classe 1
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)

        axe.plot(fpr, tpr, label=f"{set_name} AUC = {roc_auc:.2f}")

    axe.plot([0, 1], [0, 1], "k--", label="Random")
    axe.set_xlabel("False Positive Rate")
    axe.set_ylabel("True Positive Rate")
    axe.set_title(f"ROC Curve - {model_name}")
    axe.legend()

plt.tight_layout()
plt.show()

'''

c:\Users\mohammed-amine.tanna\AppData\Local\anaconda3\envs\fft_env\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


'\n\nfig , axes = plt.subplots(1,len(datas),figsize=(20,5))\nfor  data , axe , i in zip([*datas],axes,range(len(datas))) : \n    cm = confusion_matrix(data[2][0].argmax(axis=1) , data[2][1].argmax(axis=1))\n    sns.heatmap(cm/cm.sum(axis=1,keepdims=True), annot=True,cmap=\'Blues\', ax=axe)\n    axe.set_title(f"Model {models_name[i]}")\n    axe.set_xlabel(\'Prédiction\')\n    axe.set_ylabel(\'Réel\')\n\nmodel_names = [f"Model {i}" for i in range(len(datas))]\nset_names = [\'Train\', \'Validation\', \'Test\']\nset_indices = [0, 1, 2]\n\n# Créer les sous-figures\nfig, axes = plt.subplots(1, len(datas), figsize=(20, 5))\n\n# Boucle sur les modèles et les axes\nfor data, axe, model_name in zip([*datas], axes, model_names):\n    # Pour chaque ensemble (train, val, test)\n    for idx, set_name in zip(set_indices, set_names):\n        y_true = data[idx][0].argmax(axis=1)\n        y_score = data[idx][1][:, 1]  # Probabilité de la classe 1\n        fpr, tpr, _ = roc_curve(y_true, y_score)\n     

## Prediction par rapport au motif

In [46]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Données brutes
TP = 41                              #45
FN = 10
FP = 1
TN = 1000  

# Matrice brute
cm = np.array([[TP, FN],
               [FP, TN]])

# Pourcentages par ligne
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Labels : valeur + pourcentage
labels = np.array([[f"{val}\n({pct:.1f}%)" if not np.isnan(pct) else f"{val}\n(N/A)"
                    for val, pct in zip(row_val, row_pct)]
                   for row_val, row_pct in zip(cm, cm_percent)])

# Affichage avec les couleurs selon les pourcentages
plt.figure(figsize=(10, 6))
sns.heatmap(cm_percent, annot=labels, fmt="", cmap="Blues", cbar=True,
            xticklabels=["Prédit : Joint", "Prédit : Pas joint"],
            yticklabels=["Réel : Joint", "Réel : Pas joint"],
            vmin=0, vmax=100)  # 0% à 100%

plt.title("Matrice de confusion (motifs) (seuil 0.38)")
plt.ylabel("Réel")
plt.xlabel("Prédit")
plt.show()
